In [1]:
! pip install bs4 requests pandas certifi

In [2]:
import argparse
import json
import pprint
import requests
import sys
import urllib
import pandas as pd

In [3]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
    from urllib3.util.retry import Retry
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [4]:
# User parameters 

API_KEY= '_ic3HrSmO1EOK27t_faXzYg_BEVVM49cRaYBzlH3F1v7b4Vh3EstGjeo34DtMLMqT0gbtId41REkgK65T_8TA1Iq5rGlBQ08sSTOfR_IHqk6dUh2XsY0I_gmOzPcZHYx'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'
SEARCH_LIMIT = 3

In [7]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
                        type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location',
                        default=DEFAULT_LOCATION, type=str,
                        help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except HTTPError as error:
        sys.exit(
            'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                error.code,
                error.url,
                error.read(),
            )
        )



In [15]:
term = 'coffee'
location = 'Baltimore, MD'
result = query_api(term, location)


Querying https://api.yelp.com/v3/businesses/search ...
3 businesses found, querying business info for the top result "P8J8-C1zRt2m4ai6REASkw" ...
Querying https://api.yelp.com/v3/businesses/P8J8-C1zRt2m4ai6REASkw ...
Result for business "P8J8-C1zRt2m4ai6REASkw" found:
{ 'alias': 'ceremony-coffee-roasters-mt-vernon-baltimore',
  'categories': [{'alias': 'coffeeroasteries', 'title': 'Coffee Roasteries'}],
  'coordinates': {'latitude': 39.295685, 'longitude': -76.618533},
  'display_phone': '(443) 835-1659',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '1800',
                           'is_overnight': False,
                           'start': '0700'},
                         { 'day': 1,
                           'end': '1800',
                           'is_overnight': False,
                           'start': '0700'},
                         { 'day': 2,
                           

In [16]:
if __name__ == "__main__":
    business_names = ['ceremony-coffee-roasters','onedo-coffee-roasters']

    results = query_api(business_names, 'baltimore,MD')

    for result in results:
        print(u'{0} - {1} ({2}, {3})'.format(
            result['name'], result['address'], result['latitude'], result['longitude']))

AttributeError: 'list' object has no attribute 'replace'